# TODO:
1. include real-time sim of data gen using program time

2. programatic plotting based on selected parameters

3. look at edge points of prints for laser power

In [2]:
import base64
import io
import datetime
import json

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from dash import Dash, html, dcc, Input, Output, callback, State, dash_table, no_update, ctx, MATCH, ALL
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc

In [ ]:
dir = 'data/Wall_4mm_400mm_LH0.225mm_MFR4.2_delay_1018-24-9-26-14-32-16'
local_filename = 'Transducer_4mm_400mm_LH0.225mm_move_over_for_delay_1018-24-9-26-14-32-16.txt'
df = pd.read_csv(f'{dir}/{local_filename}', skiprows=[1], skipfooter=38, engine='python')  
df.columns = df.columns.str.strip()
df = df.sort_index(axis=1)
df.shape

(83081, 82)

In [ ]:
df.to_dict('records')

In [ ]:
print(df.to_json(orient="split", date_format="iso"))

In [ ]:
datasets = {
    "exp_df": exp_df.to_json(orient="split", date_format="iso"),
    "theo_combo_df": theo_combo_df.to_json(orient="split", date_format="iso"),
    "zcs": zcs.to_json(orient="split", date_format="iso"),
    "fcm_centers": fcm_centers.to_json(orient="split", date_format="iso"),
}

return json.dumps(datasets)

datasets = json.loads(jsonified_data)
exp_df = pd.read_json(datasets["exp_df"], orient="split")
theo_combo_df = pd.read_json(datasets["theo_combo_df"], orient="split")
zcs = pd.read_json(datasets["zcs"], orient="split")
fcm_centers = pd.read_json(datasets["fcm_centers"], orient="split")

In [ ]:
df.columns.values

In [196]:
DCG_LOGO = 'https://engineering.jhu.edu/dcg/wp-content/uploads/2021/02/JH-WSE-DCG_Logo.png'
# PNNL_LOGO = 'https://upload.wikimedia.org/wikipedia/en/thumb/1/17/Pacific_Northwest_National_Laboratory_logo.svg/1920px-Pacific_Northwest_National_Laboratory_logo.svg.png'
MAG_GLASS_ICON = 'https://media.istockphoto.com/id/1249867007/vector/analytics-analysis-statistics-searching-gray-icon.jpg?s=612x612&w=0&k=20&c=Yt4RBnpog9OU1uPu9LVONX69bxsdS_HjeHNP6CnFRYs='
PNNL_LOGO = 'https://www.pnnl.gov/sites/default/files/styles/hero_1600x1200/public/media/image/AT%20SCALE%20Hero%20Image.png?h=08b866d1&itok=4Q-utOef'

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME])

# offcanvas = html.Div(
#     [
#         dbc.Button(
#             [
#                 html.I(className="fa-solid fa-filter"),
#                 ' Filter'
#             ], 
#             id='open-offcanvas',
#             color='secondary',
#             outline=True,
#             n_clicks=0,
#             class_name="me-1"
#         ),
#         dbc.Offcanvas(
#             html.Div([
#                 html.P('Drag the sliders to select specific ranges for plotting. Static values are shown as badges.'),
#                 dbc.ListGroup(id='filter_list', flush=False),
#             ]),
#             id='offcanvas',
#             title='Filter Data Values',
#             is_open=False,
#             backdrop=True
#         ),
#     ]
# )

app.layout = dbc.Container(
    [
        dbc.Row([
            dbc.Col(html.Img(src=DCG_LOGO, 
                             style={'width': '100%'}), 
                    width={'size': 3}),
            dbc.Col(html.H1('AT-SCALE Digital Twin Dashboard', style={'textAlign':'center', 'font-weight': 'bold'}), key='test', id='animated-text', className="animate__animated animate__fadeInUp animate__faster",),
            dbc.Col(html.Img(src=PNNL_LOGO, 
                             style={'width': '350px', 'height': '150px', 'object-fit': 'cover'}), 
                    width={'size': 3})
        ], justify="center", align='center'),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dbc.Button(
                            [
                                html.I(className="fa-solid fa-server"),
                                ' Server'
                            ], 
                            id='local_data',
                            className="me-1",
                            color='primary',
                            n_clicks=0
                        ),
                        dbc.Button(
                            [
                                html.I(className="fa-solid fa-upload"),
                                ' Upload'
                            ],
                            id="collapse-button",
                            className="me-1",
                            color="info",
                            n_clicks=0,
                        ),
                        dbc.Button(
                            [
                                html.I(className="fa-solid fa-filter"),
                                ' Filter'
                            ], 
                            id='open-offcanvas',
                            color='secondary',
                            outline=True,
                            n_clicks=0,
                            class_name="me-1",
                            disabled=True
                        ),
                        dbc.Collapse(
                            dcc.Upload(
                                id='upload-data',
                                children=html.Div([
                                    'Drag and Drop or ',
                                    html.A('Select Files')
                                ]),
                                style={
                                    'width': '100%',
                                    'height': '60px',
                                    'lineHeight': '60px',
                                    'borderWidth': '1px',
                                    'borderStyle': 'dashed',
                                    'borderRadius': '5px',
                                    'textAlign': 'center',
                                    'margin': '10px'
                                },
                                multiple=False
                            ),
                            id="collapse",
                            is_open=False,
                        ),
                        dbc.Offcanvas(
                            html.Div([
                                html.P('Drag the sliders to select specific ranges for plotting. Static values are shown as badges.'),
                                dbc.ListGroup(id='filter_list', flush=False),
                            ]),
                            id='offcanvas',
                            title='Filter Data Values',
                            is_open=False,
                            backdrop=True
                        ),
                    ],  
                    width="auto"
                ),
                # dbc.Col(dcc.Loading(html.Div(id='output-data-upload')))
            ],
            class_name='text-center',
            align='center',
            justify="center",
        ),
        dbc.Row(
            dbc.Col(
                [
                    html.H1('Load data to start!', className='display-3 text-muted'),
                    html.Img(src=MAG_GLASS_ICON, 
                             style={'width': '13.33%', 'animation-name': 'fadeInUp'},
                             className="animate__animated animate__fadeInUp animate__slower"),
                ]
            ),
            id='tutorial',
            class_name='text-center',
            align='center',
            justify="center",
            # style={'background-color': '#002D72'}
        ),
        dbc.Row(dbc.Col(dcc.Loading(html.Div(id='output-data-upload', style={'padding-top': '10px', 'padding-bottom': '10px'})))),
        html.Div([
            dbc.Row(
                [
                    # dbc.Row(
                    #     [
                    #         dbc.Col(
                    #             offcanvas,
                    #             width='auto',
                    #             align="end"
                    #         ),
                    #         dbc.Col(
                    #             html.Div([
                    #                 '3D Scatter Color',
                    #                 dcc.Dropdown(id='scatter_color_name', value='Program Time (s)')
                    #             ]),
                    #             width='6',
                    #             align="end"
                    #         )
                    #     ]
                    # ),
                    # dcc.Loading([
                    #     dcc.Store(id="store"),
                    #     dcc.Graph(id='3d-scatter', style={'display': 'none'})
                    # ], type='graph'),
                    dbc.Col(
                        dbc.Card(
                            [
                                dbc.CardBody(
                                    [
                                        html.H4("XYZ Scatter Plot", className="card-title"),
                                        # html.H6("Marker Color", className="card-subtitle"),
                                        'Marker Color',
                                        dcc.Dropdown(id='scatter_color_name', value='Program Time (s)', style={'width':'22rem'}),
                                        dcc.Loading([
                                            dcc.Store(id="store"),
                                            dcc.Graph(id='3d-scatter', style={'display': 'none'})
                                        ], type='graph'),
                                    ]
                                ),
                            ],
                            class_name="shadow-sm p-3 mb-5 bg-white rounded"
                            # style={"width": "18rem"},
                        ),
                        md=6
                    ),
                    dbc.Col(
                        dbc.Card(
                            [
                                dbc.CardBody(
                                    [
                                        html.H4("2D Scatter Plot", className="card-title"),
                                        dbc.Row([
                                            dbc.Col(
                                                [
                                                    'X-Axis',
                                                    dcc.Dropdown(id='xaxis_column_name', value='Program Time (s)'),
                                                    dcc.RadioItems(
                                                        ['Linear', 'Log'],
                                                        'Linear',
                                                        id='crossfilter-xaxis-type',
                                                        labelStyle={'display': 'inline-block', 'marginTop': '5px', 'marginRight': '5px'}
                                                    )
                                                ],
                                                align="end"
                                            ),
                                            dbc.Col(
                                                [
                                                    'Y-Axis',
                                                    dcc.Dropdown(id='yaxis_column_name', value='SLED (J/mm^2)'),
                                                    dcc.RadioItems(
                                                        ['Linear', 'Log'],
                                                        'Linear',
                                                        id='crossfilter-yaxis-type',
                                                        labelStyle={'display': 'inline-block', 'marginTop': '5px', 'marginRight': '5px'}
                                                    )
                                                ],
                                                align="end"
                                            ),
                                        ]),
                                        dcc.Loading(
                                            dcc.Graph(id='graph-content', style={'display': 'none'}), type='graph'
                                        )
                                    ]
                                )
                            ],
                            class_name="shadow-sm p-3 mb-5 bg-white rounded"
                        ),
                        md=6
                    )
                ], 
            ),
        ], 
        id='graph-container', 
        style={
            # 'background-color': '#002D72',
            'display': 'none'
            })
    ],
    fluid=True,
    # style={
    #     'background-image':'url("https://engineering.jhu.edu/MCP/wp-content/uploads/2021/03/architecture-VCRJZ7R_blue-scaled.jpg?id=4396")',
    #     'background-size': 'cover',
    #     'background-repeat': 'no-repeat',
    #     'background-position': 'center center'
    # }
)

# style={'width': '49%', 'display': 'inline-block', 'vertical-align': 'top'}

@callback(    
    Output("open-offcanvas", "disabled"),
    Input('store', 'data')
)
def toggle_filter_button_disabled(data):
    if data:
        return False
    
    return True

@callback(
    Output("offcanvas", "is_open"),
    Input("open-offcanvas", "n_clicks"),
    [State("offcanvas", "is_open")],
)
def toggle_offcanvas(n1, is_open):
    if n1:
        return not is_open
    
    return is_open

@app.callback(
    Output("collapse", "is_open"),
    [Input("collapse-button", "n_clicks")],
    [State("collapse", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@callback(    
    Output("tutorial", "style"),
    Input('store', 'data')
)
def toggle_tutorial(data):
    if data:
        return {'display': 'none'}
    
    return {}

def parse_contents(contents):
    if contents is None:
        raise PreventUpdate

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    return decoded

def load_df(filename, decoded):
    try:
        if 'csv' in filename:
            # Assume a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        else:
            # Assume a text file formatted by the DED printer
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')), skiprows=[1], skipfooter=38, engine='python') 
    except Exception as e:
        print(e)
        return no_update

    return df

@callback(
        Output('store', 'data'),
        Input('local_data', 'n_clicks'),
        Input('upload-data', 'contents'),
        State('upload-data', 'filename'),
)
def update_store(local_click, contents, filename):
    triggered_id = ctx.triggered_id

    if triggered_id == 'local_data':
        return local_store(local_click)
    elif triggered_id == 'upload-data':
        return upload_store(contents, filename)

def upload_store(contents, filename):
    if contents is None:
        return None    
    else:
        decoded = parse_contents(contents)
        dff = load_df(filename, decoded)
        dff.columns = dff.columns.str.strip()
        dff = dff.sort_index(axis=1)
        d = dff.to_dict('records')
        datastore = {
            "df": d,
            "uploaded_data": True
        }
        return datastore
    
def local_store(n_clicks):
    if n_clicks:
        # d = df.to_dict('records')
        d = {'uploaded_data': False}
        return d
    else:
        return None
    
@callback(
        Output('output-data-upload', 'children'),
        Input('store', 'data'),
        Input('local_data', 'n_clicks'),
        Input('upload-data', 'contents'),
        State('upload-data', 'filename'),
        State('upload-data', 'last_modified'),
)
def update_data_upload(stored_data, local_click, contents, filename, date):
    if stored_data is None:
        return no_update

    if stored_data['uploaded_data'] is False:
            return html.Div(
                    dbc.Container(
                        [
                            html.H3(f'File: {local_filename}', className='display-7', style={'textAlign':'left'}),
                            html.P(f'Shape: {df.shape}', className="lead")
                        ], 
                        className="py-3"
                    ),
                className='p-3 bg-body-secondary rounded-3'
            )
                
    
    elif contents is None:
        return None
    else:
        dff = pd.DataFrame(stored_data['df'])
        return html.Div(
            dbc.Container([
                    html.H3(f'File: {filename}', className='display-7', style={'textAlign':'left'}),
                    html.H5(f'Date Created: {datetime.datetime.fromtimestamp(date)}', style={'textAlign':'left'}),
                    html.P(f'Shape: {dff.shape}', className="lead")
                    # For debugging, display the raw contents provided by the web browser
                    # html.Div('Raw Content'),
                    # html.Pre(contents[0:200] + '...', style={
                    #     'whiteSpace': 'pre-wrap',
                    #     'wordBreak': 'break-all'
                    # }),
                    # html.Hr(),
                    # dash_table.DataTable(
                    #     dff.to_dict('records'),
                    #     [{'name': i, 'id': i} for i in dff.columns]
                    # ),
                ], 
                className="py-3"),
            className='p-3 bg-body-secondary rounded-3'
        )

def generate_filter_list_item(series):
    col_name = series.name
    min_val = series.min()
    max_val = series.max()
    val_range = max_val - min_val
    n_ticks = 20
    step_val = val_range/n_ticks
    # marks = {val: f'{val}:e' for val in df[col_name]}

    if val_range == 0:
        item = dbc.Badge(min_val, className="ms-1")
    else:
        item = dcc.RangeSlider(
                    id={'type': 'property_range_slider', 'index': f'{col_name}'},
                    min=min_val, 
                    max=max_val,
                    step=step_val, 
                    marks=None,
                    persistence=False,
                    persistence_type='local',
                    tooltip={
                        "placement": "bottom",
                        "always_visible": True,
                        "style": {"color": "LightSteelBlue", "fontSize": "11px"},
                        "template": "{value}"
                    },
                )
                        
    out = dcc.Loading(
        dbc.ListGroupItem(
            [
                col_name,
                item
            ]
        ),
        overlay_style={"visibility":"visible", "filter": "blur(2px)"},
        type='default'
    )

    # if val_range == 0:
    #     out = None
    # else:
    #     out = dcc.Loading(
    #         dbc.ListGroupItem(
    #             [
    #                 col_name, 
    #                 dcc.RangeSlider(
    #                     id={'type': 'property_range_slider', 'index': f'{col_name}'},
    #                     min=min_val, 
    #                     max=max_val,
    #                     step=step_val, 
    #                     marks=None,
    #                     persistence=False,
    #                     # persistence_type='local',
    #                     tooltip={
    #                         "placement": "bottom",
    #                         "always_visible": True,
    #                         "style": {"color": "LightSteelBlue", "fontSize": "11px"},
    #                         "template": "{value}"
    #                     },
    #                 )
    #             ]
    #         ),
    #         overlay_style={"visibility":"visible", "filter": "blur(2px)"},
    #         type='default'
    #     )

    return out

@callback(
        Output('filter_list', 'children'), 
        Input('store', 'data'),
)
def generate_filter_list(data):
    if data is None:
        return no_update

    if data['uploaded_data']:
        dff = pd.DataFrame(data['df'])
    else:
        dff = df.copy()

    list_items = [generate_filter_list_item(dff[col_name]) for col_name in dff.columns.values]
    return list_items

@callback(
        Input({'type': 'property_range_slider', 'index': ALL}, 'value'),
        State({'type': 'property_range_slider', 'index': ALL}, 'id'),
        Input('store', 'data'),
)
def show_filter(values, ids, data):
    # print('filters changed:', values, ids)

    # if data is None:
    #     return no_update

    # if data['uploaded_data']:
    #     dff = pd.DataFrame(data)
    # else:
    #     dff = df.copy()

    # for value, id in zip(values, ids):
    #     if value:
    #         low, high = value
    #         col_name = id['index']

    return None

@callback(
    [
        Output('scatter_color_name', 'options'),
        Output('xaxis_column_name', 'options'),
        Output('yaxis_column_name', 'options'),
    ],
    Input('store', 'data'),
)
def update_dropdowns(data):
    if data is None:
        return no_update
    
    if data['uploaded_data']:
        dff = pd.DataFrame(data['df'])
    else:
        dff = df.copy()

    return dff.columns.values, dff.columns.values, dff.columns.values


def apply_filter(df, slider_values, slider_ids):
    for value, id in zip(slider_values, slider_ids):
        if value:
            low, high = value
            col_name = id['index']
            mask = (df[col_name] >= low) & (df[col_name] <= high)
            df = df[mask]
    
    return df

@callback(
        Output('graph-container', 'style'),
        Input('store', 'data'),
)
def update_graph_contaienr(data):
    if data is None:
        return no_update
    
    return {}

@callback(
    Output('graph-content', 'figure'),
    Output('graph-content', 'style'),
    Input('store', 'data'),
    Input('xaxis_column_name', 'value'),
    Input('yaxis_column_name', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input({'type': 'property_range_slider', 'index': ALL}, 'value'),
    State({'type': 'property_range_slider', 'index': ALL}, 'id'),
)
def update_graph(data, xaxis_column_name, yaxis_column_name, xaxis_type, yaxis_type, slider_values, slider_ids):
    if data is None:
        return no_update
    
    # if (len(slider_values) > 0) and (not any(slider_values)):
    #     return no_update
    
    if data['uploaded_data']:
        dff = pd.DataFrame(data['df'])
    else:
        dff = df.copy()

    dff = apply_filter(dff, slider_values, slider_ids)

    fig = px.scatter(dff,
        x=xaxis_column_name,
        y=yaxis_column_name,
    )

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    # fig.layout.template = 'plotly_dark'

    return fig, {}

# @callback(
#     Output('scatter_color_name', 'options'),
#     Input('store', 'data'),
# )
# def update_scatter_color_dropdown(data):
#     if data is None:
#         return no_update
    
#     dff = pd.DataFrame(data)

#     return dff.columns.values

# @callback(
#     Output('yaxis_column_name', 'options'),
#     Input('store', 'data'),
# )
# def update_scatter_color_dropdown(data):
#     if data is None:
#         return no_update
    
#     dff = pd.DataFrame(data)

#     return dff.columns.values

@callback(
    Output('3d-scatter', 'figure'),
    Output('3d-scatter', 'style'),
    Input('store', 'data'),
    Input('scatter_color_name', 'value'),
    Input({'type': 'property_range_slider', 'index': ALL}, 'value'),
    State({'type': 'property_range_slider', 'index': ALL}, 'id'),
)
def update_scatter3d(data, scatter_color_name, slider_values, slider_ids,):
    if data is None:
        return no_update
    
    # if (len(slider_values) > 0) and (not any(slider_values)):
    #     return no_update

    if data['uploaded_data']:
        dff = pd.DataFrame(data['df'])
    else:
        dff = df.copy()

    dff = apply_filter(dff, slider_values, slider_ids)

    fig = px.scatter_3d(dff, 
                        x="X Position (mm)", 
                        y="Y Position (mm)", 
                        z='Z Position (mm)', 
                        color=scatter_color_name,
                        # animation_frame="Program Time (s)",
                        # range_x=[0, 6.2], range_y=[0, 9.3], range_z=[0, 8.1]
                        )
    
    fig.update_traces(marker=dict(size=2))
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, height=500, hovermode='closest')

    # fig.layout.template = 'plotly_dark'

    return fig, {}

app.run_server(jupyter_mode="tab", debug=True, use_reloader=False)

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME])

app.layout = dbc.Container(
    [
        dcc.Store(id="store"),
        dbc.Row(
            [
                dbc.Col(
                    html.Div(
                        [
                            dbc.Button(
                                'Local', 
                                id='local_data',
                                className="me-1",
                                color='primary',
                                n_clicks=0
                            ),
                            dbc.Button(
                                "Upload",
                                id="collapse-button",
                                className="me-1",
                                color="primary",
                                n_clicks=0,
                            ),
                            dbc.Collapse(
                                dcc.Upload(
                                    id='upload-data',
                                    children=html.Div([
                                        'Drag and Drop or ',
                                        html.A('Select Files')
                                    ]),
                                    style={
                                        'width': '100%',
                                        'height': '60px',
                                        'lineHeight': '60px',
                                        'borderWidth': '1px',
                                        'borderStyle': 'dashed',
                                        'borderRadius': '5px',
                                        'textAlign': 'center',
                                        'margin': '10px'
                                    },
                                    multiple=False
                                ),
                                id="collapse",
                                is_open=False,
                            )
                        ]
                    ),  
                width="auto"
                ),
                dbc.Col(dcc.Loading(html.Div(id='output-data-upload')))
            ]
        )
    ]
)

@app.callback(
    Output("collapse", "is_open"),
    [Input("collapse-button", "n_clicks")],
    [State("collapse", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

def parse_contents(contents):
    if contents is None:
        raise PreventUpdate

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    return decoded

def load_df(filename, decoded):
    print('in load_df')
    try:
        if 'csv' in filename:
            # Assume a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        else:
            # Assume a text file formatted by the DED printer
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')), skiprows=[1], skipfooter=38, engine='python') 
    except Exception as e:
        print(e)
        return no_update

    print('df laoded:', df.shape)
    return df

@callback(
        Output('store', 'data'),
        Input('local_data', 'n_clicks'),
        Input('upload-data', 'contents'),
        State('upload-data', 'filename'),
)
def update_store(local_click, contents, filename):
    triggered_id = ctx.triggered_id

    if triggered_id == 'local_data':
        print('local data')
        return local_store(local_click)
    elif triggered_id == 'upload-data':
        print('uplaod data triggered')
        data = upload_store(contents, filename)
        print('after uplaod_store')
        return data
    
def upload_store(contents, filename):
    if contents is None:
        return None    
    else:
        decoded = parse_contents(contents)
        dff = load_df(filename, decoded)
        dff.columns = dff.columns.str.strip()
        print('df sort')
        dff = dff.sort_index(axis=1)
        print('df to dict')

        d = dff.to_dict('records')
        # d = dff.to_json(orient="split", date_format="iso"),
        datastore = {
            "df": d,
            "uploaded_data": True
        }

        # print(len(d))
        # print('df add uploaded bool')
        # d['uploaded_data'] = True
        # print('upload set:', d['uploaded_data'])

        # return json.dumps(datasets)
        return datastore

def local_store(n_clicks):
    print('in local_store')
    if n_clicks:
        # d = df.to_dict('records')
        d = {'uploaded_data': False}
        return d
    else:
        return None
    
@callback(
        Output('output-data-upload', 'children'),
        Input('store', 'data'),
        Input('local_data', 'n_clicks'),
        Input('upload-data', 'contents'),
        State('upload-data', 'filename'),
        State('upload-data', 'last_modified'),
)
def update_data_upload(stored_data, local_click, contents, filename, date):
    if local_click:
            return html.Div([
                html.H3(f'File: {local_filename}'),
                html.Hr(),  # horizontal line
                dash_table.DataTable(
                    df.to_dict('records'),
                    [{'name': i, 'id': i} for i in df.columns]
                ),
            ])
    elif contents is None:
        return None
    else:
        print(stored_data['uploaded_data'])
        dff = pd.DataFrame(stored_data['df'])
        print('before render uploaded data', dff.shape)
        return html.Div([
                    html.H3(f'File: {filename}', style={'textAlign':'center'}),
                    html.H5(f'Date Created: {datetime.datetime.fromtimestamp(date)}', style={'textAlign':'center'}),

                    # For debugging, display the raw contents provided by the web browser
                    html.Div('Raw Content'),
                    html.Pre(contents[0:200] + '...', style={
                        'whiteSpace': 'pre-wrap',
                        'wordBreak': 'break-all'
                    }),
                    html.Hr(),
                    html.Div('DataFrame'),
                    dcc.Loading(dash_table.DataTable(
                        dff.to_dict('records'),
                        [{'name': i, 'id': i} for i in dff.columns]
                    ))
                    # dash_table.DataTable(
                    #     stored_data
                    # )
                ])
    
app.run_server(juypter_mode='jupyterlab', debug=True)